In [4]:
import arcpy

# Define feature classes and join field
target_fc = "scrap_feat_dup_name"
# source_fc = "scrap_feat_dup_name_web_dist"
source_fc = "scrap_feat_dup_name_whiskyMap"

join_field = "distillery_name"  # Common field between source and target

# Get field names, excluding ObjectID fields
fields = [f.name for f in arcpy.ListFields(target_fc) if f.name not in ("OBJECTID", "Shape")]

# Ensure join_field is in fields
if join_field not in fields:
    raise ValueError(f"Join field '{join_field}' not found in target feature class.")

# Prepare field lists for cursor
target_fields = [join_field] + fields  # Include join field in update cursor
source_fields = [join_field] + fields  # Include join field in search cursor

# Create a dictionary from source_fc
source_dict = {}
with arcpy.da.SearchCursor(source_fc, source_fields) as search_cursor:
    for row in search_cursor:
        source_dict[row[0]] = row[1:]  # Store all attributes (except join_field)

# Update target_fc where attributes are NULL
with arcpy.da.UpdateCursor(target_fc, target_fields) as update_cursor:
    for row in update_cursor:
        key = row[0]  # Get the join field value
        if key in source_dict:
            updated_row = list(row)  # Convert to list for editing
            source_values = source_dict[key]
            for i in range(1, len(fields)):  # Skip join field (index 0)
                if updated_row[i] is None or updated_row[i] == "":  # Check for NULL/Empty
                    updated_row[i] = source_values[i - 1]  # Assign source value

            update_cursor.updateRow(updated_row)  # Update row in target_fc

print("Update complete!")


Update complete!
